# Imports:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import chi2

from tqdm import tqdm, trange
import random

In [2]:
import matplotlib.pyplot as plt
import pickle
import statsmodels.api as sm
from scipy.special import expit, logit

# Hyperparameters:

In [3]:
pop_size = 100 * 1_000
sample_size = 30
agg_iters = 100_000

beta = 1

## for biased sampling mechanism:
sample_probability_centering = 0.8
sample_probability_bias_factor = 0.1

In [4]:
## for rng generator:
pop_number = 1

# Run:

In [5]:
rand_generator = np.random.default_rng(seed=333 * pop_number)

In [6]:
assert pop_size % 100 == 0
base_x = [i for i in range(-50, 0)] + [i for i in range(1, 51)]
x = pd.Series(base_x * (pop_size // 100))
normal_stdev = 30

In [7]:
np.var(x)

np.float64(858.5)

In [8]:
fitted_betas = []
fitted_FI = []
fitted_correlations = []
realized_sample_sizes = []

Create the simulated data (x, y):

In [9]:
y = x * beta + rand_generator.normal(size=pop_size) * normal_stdev

data = pd.concat([y, x], axis=1)

data.columns = ['y', 'x']

fitted_beta = sm.OLS(data['y'], data['x']).fit().params.iloc[0]

data['g_star'] = data['x'] * (data['y'] - data['x'] * fitted_beta)

In [10]:
marginal_probabilities = expit(logit(sample_probability_centering) + \
                               sample_probability_bias_factor * (data['x'].abs() - 25.5))

# run:

In [11]:
for _ in tqdm(range(agg_iters)):
    srs_sample_indices = pd.Series(random.sample(range(pop_size), sample_size))
    srs_sample_indices = srs_sample_indices.sort_values().reset_index(drop=True)

    # reset the sample
    data['r0'] = 0
    data['r'] = 0
    data['r1'] = 0
    data['r2'] = 0
    
    data.loc[srs_sample_indices, 'r0'] = 1
    full_sampled_data = data[data['r0'] == 1]

    marginal_probabilities = expit(logit(sample_probability_centering) + \
                                   sample_probability_bias_factor * (full_sampled_data['x'].abs() - 25.5))
    biased_sample_indices = marginal_probabilities.index[rand_generator.binomial(n=1, p = marginal_probabilities) == 1]

    data.loc[biased_sample_indices, 'r'] = 1

    biased_sample_indices_size = len(biased_sample_indices)
    realized_sample_sizes.append(biased_sample_indices_size)


    srs_sample = data[data['r0'] == 1].copy()
    biased_sample = data[data['r'] == 1].copy()

    srs_fitted_beta = sm.OLS(srs_sample['y'], srs_sample['x']).fit().params.iloc[0]
    biased_fitted_beta = sm.OLS(biased_sample['y'], biased_sample['x']).fit().params.iloc[0]

    fitted_betas.append(pd.Series([fitted_beta, srs_fitted_beta, biased_fitted_beta]))

    fitted_FI.append(pd.Series([np.mean(srs_sample['x'] ** 2), np.mean(biased_sample['x'] ** 2)]))

    fitted_correlations.append(data[['g_star', 'r0', 'r']].corr().iloc[0, 1:])

100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [09:36<00:00, 173.45it/s]


Clean up the data:

In [12]:
fitted_betas = pd.concat(fitted_betas, axis=1).T
fitted_FI = pd.concat(fitted_FI, axis=1).T
fitted_correlations = pd.concat(fitted_correlations, axis=1).T.reset_index(drop=True)

In [13]:
fitted_FI.columns = ['srs', 'biased']
fitted_betas.columns = ['population', 'srs', 'biased']
fitted_correlations.columns = ['srs', 'biased']

In [14]:
realized_sample_sizes = pd.Series(realized_sample_sizes)

In [15]:
fitted_FI.head()

,srs,biased
0,999.333333,1101.038462
1,681.666667,820.086957
2,862.433333,1031.217391
3,819.200000,962.650000
4,974.266667,1199.130435


In [16]:
fitted_FI.tail()

,srs,biased
99995,889.833333,1097.666667
99996,986.966667,1133.720000
99997,806.000000,1197.055556
99998,787.733333,969.300000
99999,889.866667,1220.600000


In [17]:
fitted_FI_and_corr = fitted_FI.join(fitted_correlations, lsuffix = '_fi', rsuffix = '_corr')

fitted_FI_and_corr.head()

,srs_fi,biased_fi,srs_corr,biased_corr
0,999.333333,1101.038462,0.001389,0.001919
1,681.666667,820.086957,0.001253,0.000489
2,862.433333,1031.217391,0.002659,0.002825
3,819.200000,962.650000,0.002268,0.002823
4,974.266667,1199.130435,-0.002265,-0.002186


In [18]:
np.mean(data['x'] ** 2)

np.float64(858.5)

In [19]:
Jeee = np.mean(data['x'] ** 2)

fitted_FI_and_corr['srs_samp_elasticity'] = fitted_FI_and_corr.eval(f'srs_fi / {Jeee}')
fitted_FI_and_corr['biased_samp_elasticity'] = fitted_FI_and_corr.eval(f'biased_fi / {Jeee}')

fitted_FI_and_corr.to_pickle('sample_by_x_bias.pkl')